In [2]:
import pandas as pd
import numpy as np
import os
import json
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

root_path = "/workspace/med_proj"
# 读取数据
file_path = f'{root_path}/data/体温.xlsx'
data = pd.read_excel(file_path)

# 数据预处理，删除无用列，选择是否删除第四天的数据，删除空行，重命名列名，添加ID列
data = data.iloc[:, :-2]

time_during = "48h"
assert time_during in ["48h", "72h", "96h"]

output_dir = f'{root_path}/results/dataprocess/{time_during}'
if not os.path.exists(output_dir):
    os.makedirs(output_dir, exist_ok=True)

if time_during == "48h":
    fourth_day_columns = [col for col in data.columns if '第四天' in col or '第三天' in col]
elif time_during == "72h":
    fourth_day_columns = [col for col in data.columns if '第四天' in col]
else:
    fourth_day_columns = []
    
data = data.drop(columns=fourth_day_columns)
data = data.replace('\\', np.nan)
data = data.dropna(thresh=data.shape[1] - 3)
data = data.rename(columns={'Unnamed: 0': 'Name'})
data['ID'] = range(1, len(data) + 1)

# 保存 ID 映射表
name_id_mapping = name_id_mapping = dict(zip(data['Name'], data['ID']))
if time_during == "48h":
    name_id_mapping['曹玉儿'] = 59
name_id_mapping = dict(sorted(name_id_mapping.items(), key=lambda item: item[1]))


json_file_path = f'{output_dir}/name_id_mapping.json'
with open(json_file_path, 'w') as file:
    json.dump(name_id_mapping, file, ensure_ascii=False, indent=4)

output_excel_path = f'{output_dir}/temperature_v1.xlsx'
data.to_excel(output_excel_path, index=False)

# 多重差补法填充缺失值
new_data = data.drop(columns=['Name', "ID"])
imputer = IterativeImputer()
imputed_data = imputer.fit_transform(new_data)
imputed_df_sklearn = pd.DataFrame(imputed_data, columns=new_data.columns)

# 保存填充后的数据
output_imputed_df_path = f'{output_dir}/temperature_v2.xlsx'
imputed_df_sklearn.to_excel(output_imputed_df_path, index=False)


# 讲数据转化为长格式
data = imputed_df_sklearn.reset_index().rename(columns={'index': 'ID'})
data["ID"] = data["ID"]+1
melted_data = pd.melt(data, id_vars=['ID'], var_name='Time', value_name='Temperature')
time_mapping = {time: idx for idx, time in enumerate(melted_data['Time'].unique(), start=1)}
melted_data['Time'] = melted_data['Time'].map(time_mapping)
melted_data_sorted = melted_data.sort_values(by=['ID', 'Time'])

output_long_data_path = f'{output_dir}/temperature_v3.xlsx'
melted_data_sorted.to_excel(output_long_data_path, index=False)

if time_during == "48h":
    new_columns = ['t.' + str(i) for i in range(1, 13)]
elif time_during == "72h":
    new_columns = ['t.' + str(i) for i in range(1, 19)]
else:
    new_columns = ['t.' + str(i) for i in range(1, 25)]

imputed_df_sklearn.columns = new_columns

imputed_df_sklearn.to_excel(f"{output_dir}/wide_data.xlsx", index=False)
imputed_df_sklearn.to_csv(f"{output_dir}/wide_data.csv", index=False)


/tmp/ipykernel_958696/1834230997.py:31: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data = data.replace('\\', np.nan)


#### 归一化

In [2]:
df = imputed_df_sklearn
print(df.max().max(), df.min().min(), df.max().max() - df.min().min())
df_normalized = (df - df.min().min()) / (df.max().max() - df.min().min()) * 100   # -34.2/7.799999999999997
df_normalized.to_csv(f"{output_dir}/wide_data_norm.csv", index=False)

# 添加 ID 列
df_normalized.insert(0, 'ID', range(1, 1 + len(df_normalized)))
df_normalized.to_csv(f"{output_dir}/wide_data_norm_id.csv", index=False)

42.0 34.3 7.700000000000003


In [27]:
# 使用 48h 的结果：42.0 34.3 7.700000000000003

# 使用 72h 的结果：42.0 34.2 7.799999999999997

# 使用 96h 的结果：40.8 34.3 6.5
